In [5]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [9]:
import pyspark.sql.functions as F

flights_df = spark.read.csv('file:///home/talentum/test-jupyter/test/PairRdd/flights.csv')
flight_orig_dest_df = flights_df.select(
    F.col("_c12").alias("origin"),
    F.col("_c13").alias("destination")
)
print("Flight Origin-Destination DataFrame:")
flight_orig_dest_df.show(5)

Flight Origin-Destination DataFrame:
+------+-----------+
|origin|destination|
+------+-----------+
|   IAD|        TPA|
|   IND|        BWI|
|   IND|        JAX|
|   IND|        LAS|
|   IND|        PHX|
+------+-----------+
only showing top 5 rows



In [11]:
cities_df = flights_df.select(
    F.col("_c12").alias("airport_code")
).union(
    flights_df.select(
        F.col("_c13").alias("airport_code")
    )
).distinct()

print("Cities DataFrame:")
cities_df.show(5)

Cities DataFrame:
+------------+
|airport_code|
+------------+
|         BGM|
|         PSE|
|         DLG|
|         INL|
|         MSY|
+------------+
only showing top 5 rows



In [12]:
orig_join_df = flight_orig_dest_df.join(
    cities_df,
    flight_orig_dest_df.origin == cities_df.airport_code
)
print("Origin Join DataFrame:")
orig_join_df.show(5)

Origin Join DataFrame:
+------+-----------+------------+
|origin|destination|airport_code|
+------+-----------+------------+
|   BGM|        DTW|         BGM|
|   BGM|        DTW|         BGM|
|   BGM|        DTW|         BGM|
|   BGM|        DTW|         BGM|
|   BGM|        DTW|         BGM|
+------+-----------+------------+
only showing top 5 rows



In [13]:
cities_count_df = flight_orig_dest_df.groupBy("destination").count().orderBy("count", ascending=False)
print("Cities Count DataFrame:")
cities_count_df.show(5)

Cities Count DataFrame:
+-----------+------+
|destination| count|
+-----------+------+
|        ATL|122096|
|        ORD| 99709|
|        DFW| 81532|
|        DEN| 71347|
|        LAX| 63682|
+-----------+------+
only showing top 5 rows



In [16]:
from pyspark.sql.types import *

delays_df = flights_df.select(
    F.col("_c5").alias("flight_num"),
    F.col("_c11").cast(IntegerType()).alias("delay")
).filter(F.col("delay") > 15)
print("Delays DataFrame:")
delays_df.show(5)


Delays DataFrame:
+----------+-----+
|flight_num|delay|
+----------+-----+
|        WN|   25|
|        WN|   67|
|        WN|   87|
|        WN|   29|
|        WN|   82|
+----------+-----+
only showing top 5 rows



In [20]:
max_delays_df = delays_df.groupBy("flight_num").agg(F.max("delay").alias("max_delay")).orderBy("max_delay", ascending=False)
print("Maximum Delays DataFrame:")
max_delays_df.show(5)

Maximum Delays DataFrame:
+----------+---------+
|flight_num|max_delay|
+----------+---------+
|        NW|     2457|
|        AA|     1521|
|        MQ|     1295|
|        UA|     1268|
|        9E|     1099|
+----------+---------+
only showing top 5 rows



In [41]:
plane_data_df = spark.read.csv(
    'file:///home/talentum/test-jupyter/test/PairRdd/plane-data.csv',
    header=True, 
    inferSchema=True
)
print("Total plane records:", plane_data_df.count())

Total plane records: 5029


In [45]:
filtered_df = plane_data_df.filter("type IS NOT NULL")

filtered_df.count()

4480